In [5]:
import sys

In [6]:
!{sys.executable} -m pip install yfinance

In [7]:
from datetime import datetime, timezone
from datetime import datetime, timedelta
import time
import yfinance as yf
import pandas as pd

# Function to convert UNIX timestamp (in milliseconds) to date string (YYYY-MM-DD)
def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    epoch_secs = timestamp / 1000  # Convert milliseconds to seconds
    dt_object = datetime.fromtimestamp(epoch_secs, tz=timezone.utc)  # Ensure UTC timezone
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

# Function to convert date string (YYYY-MM-DD) to UNIX timestamp in milliseconds
def date_to_timestamp(date):
    """
    Convert a string of the form YYYY-MM-DD to the timestamp of that day at 05:00:00 UTC.
    """
    date_obj = datetime.strptime(date, '%Y-%m-%d').replace(hour=5, minute=0, second=0, microsecond=0, tzinfo=timezone.utc)
    timestamp_milliseconds = int(date_obj.timestamp() * 1000)  # Convert to milliseconds
    
    return timestamp_milliseconds

# Example usage with yfinance
ticker = "AAPL"
start_date = "2024-01-01"
end_date = "2024-02-01"

# Fetch stock data from Yahoo Finance
df = yf.download(ticker, start=start_date, end=end_date)

# Convert timestamps to date format
df.index = df.index.strftime('%Y-%m-%d')

df.head()


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2024-01-02,184.532074,187.315366,182.792518,186.033057,82488700
2024-01-03,183.150375,184.770652,182.335262,183.120556,58414500
2024-01-04,180.824356,181.997307,179.800504,181.062914,71983600
2024-01-05,180.098694,181.669266,179.094727,180.903872,62303300
2024-01-08,184.452560,184.492330,180.416793,181.003268,59144500


In [8]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all available stock data from Yahoo Finance 
    for a given stock between start_date and end_date.
    Dates are in the format YYYY-MM-DD.
    """

    # Fetch stock data
    df = yf.download(ticker, start=start_date, end=end_date)

    # Reset index to get the Date as a column
    df.reset_index(inplace=True)

    # Rename columns to match the original function (except VWAP)
    df.rename(columns={
        'Date': 'Date',
        'Open': 'Open',
        'High': 'High',
        'Low': 'Low',
        'Close': 'Close',
        'Volume': 'Volume'
    }, inplace=True)

    # Convert the Date column to timestamps (milliseconds)
    df['Timestamp'] = df['Date'].astype('int64') // 10**6  # Convert to milliseconds

    # Rearrange columns (excluding VWAP)
    # Add this line inside the function to compute VWAP before rearranging columns
    df['Vwap'] = (df['High'] + df['Low'] + df['Close']) / 3
    df = df[['Timestamp', 'Date', 'Close', 'Volume', 'Open', 'High', 'Low', 'Vwap']]


    return df


In [9]:
# Example of single_stock_table
df = single_stock_table('NVDA', '2022-08-01', '2025-02-12')
df.head()

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,Date,Close,Volume,Open,High,Low,Vwap
Ticker,,,NVDA,NVDA,NVDA,NVDA,NVDA,
0,1659312000000,2022-08-01,18.417414,476469000,18.158744,18.821896,17.966990,18.402100
1,1659398400000,2022-08-02,18.502300,489527000,18.098818,18.913774,18.068855,18.494977
2,1659484800000,2022-08-03,18.868830,418146000,18.160737,18.943735,18.113796,18.642120
3,1659571200000,2022-08-04,19.190422,409652000,18.824891,19.249347,18.736004,19.058591
4,1659657600000,2022-08-05,18.964712,386068000,18.785940,19.204405,18.642125,18.937081


In [10]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing stock info for multiple specified tickers on a given date.
    Date is a string in the format YYYY-MM-DD.
    Dataframe is sorted by trade volume.
    """

    # Define a list of commonly traded stocks (can be expanded)
    ticker_list = ["AAPL", "GOOGL", "AMZN", "MSFT", "TSLA", "NVDA", "META", "NFLX"]

    # Convert date format for Yahoo Finance
    formatted_date = datetime.strptime(date, '%Y-%m-%d')
    next_day = formatted_date + timedelta(days=1)  # yfinance requires an end date

    # Fetch stock data
    data = yf.download(ticker_list, start=formatted_date.strftime('%Y-%m-%d'), 
                       end=next_day.strftime('%Y-%m-%d'), interval="1d")

    # Extract relevant fields
    df = pd.DataFrame({
        "Close": data["Close"].iloc[0],
        "Volume": data["Volume"].iloc[0],
        "Open": data["Open"].iloc[0],
        "High": data["High"].iloc[0],
        "Low": data["Low"].iloc[0]
    })

    # Sort by trading volume
    df = df.sort_values("Volume", ascending=False)

    return df


In [11]:
df = multiple_stock_table("2024-02-20")
df

[*********************100%***********************]  8 of 8 completed


,Close,Volume,Open,High,Low
Ticker,,,,,
NVDA,69.432320,704833000,71.926608,71.935608,67.714804
TSLA,193.759995,104545800,196.130005,198.600006,189.130005
AAPL,180.706741,53665600,180.935656,181.572648,179.154075
AMZN,167.080002,41980300,167.830002,168.710007,165.740005
GOOGL,140.610275,25144700,139.155557,141.566814,138.986173
MSFT,399.738190,24307900,400.184762,401.425291,394.994408
META,469.954803,18015500,467.932530,474.367938,464.784551
NFLX,575.130005,3124200,580.179993,586.000000,569.659973


In [12]:
def fetch_macd(stock_ticker, start_date, end_date, short_window=12, long_window=26, signal_window=9):
    """
    Fetch MACD data for a given stock using yfinance.

    - If MACD > Signal Line → Upward trend (potential buy signal)
    - If MACD < Signal Line → Downward trend (potential sell signal)

    Parameters:
    - stock_ticker: Stock symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - short_window: Short-term EMA period (default=12)
    - long_window: Long-term EMA period (default=26)
    - signal_window: Signal line EMA period (default=9)
    """

    # Fetch stock data from Yahoo Finance
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Compute short-term and long-term exponential moving averages (EMA)
    df['Short_EMA'] = df['Close'].ewm(span=short_window, adjust=False).mean()
    df['Long_EMA'] = df['Close'].ewm(span=long_window, adjust=False).mean()

    # Calculate MACD Line
    df['MACD Value'] = df['Short_EMA'] - df['Long_EMA']

    # Calculate Signal Line (9-day EMA of MACD)
    df['Signal'] = df['MACD Value'].ewm(span=signal_window, adjust=False).mean()

    # Convert Date index to a column and format
    df.reset_index(inplace=True)
    df.rename(columns={"Date": "Timestamp"}, inplace=True)
    df['Timestamp'] = df['Timestamp'].astype(str)  # Convert Timestamp to string for consistency

    # Select relevant columns
    df = df[['Timestamp', 'MACD Value', 'Signal']]

    return df

In [13]:
df_macd = fetch_macd("AAPL", "2023-01-01", "2024-02-20")
df_macd.head()

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,MACD Value,Signal
Ticker,,,
0,2023-01-03,0.000000,0.000000
1,2023-01-04,0.101724,0.020345
2,2023-01-05,0.074596,0.031195
3,2023-01-06,0.415225,0.108001
4,2023-01-09,0.719162,0.230233


In [14]:
def stock_proc(ticker, start_date, end_date, n):
    """
    Fetch stock data for a given ticker between start_date and end_date,
    and compute the Price Rate of Change (PROC) over the last 'n' days.
    Automatically fixes incorrect date order.

    Parameters:
    - ticker: Stock symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - n: Number of days for PROC calculation

    Returns:
    - A DataFrame with the latest available PROC value
    """

    start_date, end_date = min(start_date, end_date), max(start_date, end_date)

    df = yf.download(ticker, start=start_date, end=end_date, interval="1d")

    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() for col in df.columns.values]


    close_cols = [col for col in df.columns if "Close" in col]
    
    close_col = close_cols[0]  

    df["Close"] = df[close_col].astype(float)

    df["Close_n_days_ago"] = df["Close"].shift(n).astype(float)  

    df["PROC"] = ((df["Close"] - df["Close_n_days_ago"]) / df["Close_n_days_ago"]) * 100

    df["PROC"] = df["PROC"].replace([float("inf"), -float("inf")], None)
    df["PROC"] = df["PROC"].fillna(0)

    if not df.empty:
        latest_data = df.iloc[-1][["Close", "Close_n_days_ago", "PROC"]].to_dict()
    else:
        latest_data = {"Close": None, "Close_n_days_ago": None, "PROC": None}

    result = pd.DataFrame({
        "Ticker": [ticker],
        "Current Close": [latest_data["Close"]],
        f"Close {n} Days Ago": [latest_data["Close_n_days_ago"]],
        f"PROC ({n} days)": [latest_data["PROC"]]
    })

    return result

In [15]:
df_proc = stock_proc("AAPL", "2023-01-01", "2024-02-20", n=10)
df_proc

[*********************100%***********************]  1 of 1 completed


,Ticker,Current Close,Close 10 Days Ago,PROC (10 days)
0,AAPL,181.453201,184.740829,-1.779589


In [16]:
def fetch_rsi(stock_ticker, start_date, end_date, window=14):
    """
    Fetches the Relative Strength Index (RSI) for a given stock ticker using yfinance.
    
    - RSI > 70 → Overbought (potential price drop)
    - RSI < 30 → Oversold (potential price increase)
    
    Parameters:
    - stock_ticker: Ticker symbol (e.g., "AAPL")
    - start_date, end_date: Date range (YYYY-MM-DD)
    - window: Period for RSI calculation (default = 14)

    Returns:
    - DataFrame with RSI values
    """

    # Fetch stock data
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Calculate price change
    df["Price Change"] = df["Close"].diff()

    # Calculate gains and losses
    df["Gain"] = df["Price Change"].apply(lambda x: x if x > 0 else 0)
    df["Loss"] = df["Price Change"].apply(lambda x: -x if x < 0 else 0)

    # Calculate rolling average gains and losses
    avg_gain = df["Gain"].rolling(window=window, min_periods=1).mean()
    avg_loss = df["Loss"].rolling(window=window, min_periods=1).mean()

    # Compute Relative Strength (RS)
    rs = avg_gain / avg_loss
    df["RSI Value"] = 100 - (100 / (1 + rs))

    # Convert Date index to a column
    df.reset_index(inplace=True)
    df.rename(columns={"Date": "Timestamp"}, inplace=True)

    # Select relevant columns
    df = df[["Timestamp", "RSI Value"]]

    return df


In [17]:
# Example of fetch_rsi
rsi_df = fetch_rsi("AAPL", '2024-12-12', '2025-01-31')
rsi_df

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,RSI Value
Ticker,,
0,2024-12-12,NaN
1,2024-12-13,100.000000
2,2024-12-16,100.000000
3,2024-12-17,100.000000
4,2024-12-18,50.410907
5,2024-12-19,57.210353
6,2024-12-20,68.775177
7,2024-12-23,70.115597
8,2024-12-24,74.265433


In [18]:
import pandas as pd
import yfinance as yf

def range_stock_data(start_date, end_date, stock_ticker):
    """
    Fetches historical stock data (High, Low, Close, Open, Volume) over a given date range.

    Parameters:
    - start_date (str): Start date in YYYY-MM-DD format.
    - end_date (str): End date in YYYY-MM-DD format.
    - stock_ticker (str): Stock symbol (e.g., "AAPL").

    Returns:
    - DataFrame with historical stock data.
    """

    # Fetch stock data from Yahoo Finance
    df = yf.download(stock_ticker, start=start_date, end=end_date, interval="1d")

    # Reset index to move Date from index to column
    df.reset_index(inplace=True)

    # Rename columns to match the Polygon API format
    df.rename(columns={"Date": "Timestamp"}, inplace=True)

    # Ensure the Timestamp column is in datetime format
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])

    # Select only the necessary columns
    df = df[["Timestamp", "Open", "High", "Low", "Close", "Volume"]]

    return df


In [19]:
df = range_stock_data("2024-01-01", "2024-02-15", "AAPL")
df

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,Open,High,Low,Close,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,2024-01-02,186.033057,187.315366,182.792518,184.532074,82488700
1,2024-01-03,183.120556,184.770652,182.335262,183.150375,58414500
2,2024-01-04,181.062914,181.997307,179.800504,180.824356,71983600
3,2024-01-05,180.903872,181.669266,179.094727,180.098694,62303300
4,2024-01-08,181.003268,184.492330,180.416793,184.452560,59144500
5,2024-01-09,182.822345,184.045000,181.639444,184.035065,42841800
6,2024-01-10,183.249796,185.287550,182.822355,185.078812,46792900
7,2024-01-11,185.426719,185.933685,182.524147,184.482391,49128400
8,2024-01-12,184.949573,185.625523,184.084771,184.810410,40444700


In [20]:
import pandas as pd
import yfinance as yf

def calculate_williams_r(stock_data, window=14):
    """
    Calculates Williams %R for the given stock data.
    
    - Above -20 → Overbought (Sell signal)
    - Below -80 → Oversold (Buy signal)
    
    Parameters:
    - stock_data: DataFrame containing "High", "Low", and "Close" columns.
    - window: The period over which Williams %R is calculated (default = 14 days).
    
    Returns:
    - DataFrame with an additional "Williams %R" column.
    """

    # Compute the highest high and lowest low over the window period
    highest_high = stock_data["High"].rolling(window=window, min_periods=1).max()
    lowest_low = stock_data["Low"].rolling(window=window, min_periods=1).min()

    # Calculate Williams %R
    stock_data["Williams %R"] = ((highest_high - stock_data["Close"]) / 
                                 (highest_high - lowest_low)) * -100

    return stock_data


In [21]:
# example of calculate_williams_r
stock_data = single_stock_table("AAPL", "2024-01-01", "2024-02-15")
df = calculate_williams_r(stock_data)
df.dropna()

[*********************100%***********************]  1 of 1 completed


Price,Timestamp,Date,Close,Volume,Open,High,Low,Vwap,Williams %R
Ticker,,,AAPL,AAPL,AAPL,AAPL,AAPL,,
0,1704153600000,2024-01-02,184.532074,82488700,186.033057,187.315366,182.792518,184.879986,-61.538487
1,1704240000000,2024-01-03,183.150375,58414500,183.120556,184.770652,182.335262,183.418763,-83.632603
2,1704326400000,2024-01-04,180.824356,71983600,181.062914,181.997307,179.800504,180.874056,-86.375642
3,1704412800000,2024-01-05,180.098694,62303300,180.903872,181.669266,179.094727,180.287562,-87.787242
4,1704672000000,2024-01-08,184.452560,59144500,181.003268,184.492330,180.416793,183.120561,-34.824613
5,1704758400000,2024-01-09,184.035065,42841800,182.822345,184.045000,181.639444,183.239836,-39.903242
6,1704844800000,2024-01-10,185.078812,46792900,183.249796,185.287550,182.822355,184.396239,-27.206578
7,1704931200000,2024-01-11,184.482391,49128400,185.426719,185.933685,182.524147,184.313408,-34.461735
8,1705017600000,2024-01-12,184.810410,40444700,184.949573,185.625523,184.084771,184.840234,-30.471556


In [22]:
def calculate_KDJ(stock_data, window_k_raw=9, window_d=3, window_k_smooth=3):
    """
    Calculates the KDJ indicators for a given stock dataset.

    - %K = Smoothed RSV (Relative Strength Value)
    - %D = Moving Average of %K
    - %J = 3 * %K - 2 * %D (momentum signal)

    Parameters:
    - stock_data: DataFrame from `single_stock_table()`
    - window_k_raw: Period for RSV calculation (default = 9 days)
    - window_d: Period for %D smoothing (default = 3 days)
    - window_k_smooth: Period for %K smoothing (default = 3 days)

    Returns:
    - DataFrame with %K, %D, and %J indicators added.
    """

    # Flatten column headers if they are multi-indexed
    if isinstance(stock_data.columns, pd.MultiIndex):
        stock_data.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in stock_data.columns]

    # Ensure the dataset contains necessary columns (matching single_stock_table())
    required_columns = {"High_AAPL", "Low_AAPL", "Close_AAPL"}
    if not required_columns.issubset(stock_data.columns):
        raise ValueError(f"Missing required columns: {required_columns - set(stock_data.columns)}. Available columns: {list(stock_data.columns)}")

    # Calculate RSV (Raw Stochastic Value)
    stock_data["H"] = stock_data["High_AAPL"].rolling(window=window_k_raw, min_periods=1).max()
    stock_data["L"] = stock_data["Low_AAPL"].rolling(window=window_k_raw, min_periods=1).min()

    # Prevent division by zero (if H == L, replace with 1 to avoid NaN)
    stock_data["RSV"] = ((stock_data["Close_AAPL"] - stock_data["L"]) /
                         (stock_data["H"] - stock_data["L"]).replace(0, 1)) * 100

    # Calculate %K as a moving average of RSV
    stock_data["%K"] = stock_data["RSV"].rolling(window=window_k_smooth, min_periods=1).mean()

    # Calculate %D as a moving average of %K
    stock_data["%D"] = stock_data["%K"].rolling(window=window_d, min_periods=1).mean()

    # Calculate %J as a momentum signal
    stock_data["%J"] = 3 * stock_data["%K"] - 2 * stock_data["%D"]

    # Drop intermediate columns
    stock_data.drop(columns=["H", "L", "RSV"], inplace=True)

    return stock_data


In [23]:
# Fetch stock data
stock_data = single_stock_table("AAPL", "2024-01-01", "2024-02-15")
df_kdj = calculate_KDJ(stock_data)
df_kdj.dropna()

[*********************100%***********************]  1 of 1 completed


,Timestamp_,Date_,Close_AAPL,Volume_AAPL,Open_AAPL,High_AAPL,Low_AAPL,Vwap_,%K,%D,%J
0,1704153600000,2024-01-02,184.532074,82488700,186.033057,187.315366,182.792518,184.879986,38.461513,38.461513,38.461513
1,1704240000000,2024-01-03,183.150375,58414500,183.120556,184.770652,182.335262,183.418763,27.414455,32.937984,16.367397
2,1704326400000,2024-01-04,180.824356,71983600,181.062914,181.997307,179.800504,180.874056,22.817756,29.564574,9.324119
3,1704412800000,2024-01-05,180.098694,62303300,180.903872,181.669266,179.094727,180.287562,14.068171,21.433461,-0.662408
4,1704672000000,2024-01-08,184.452560,59144500,181.003268,184.492330,180.416793,183.120561,30.337501,22.407809,46.196884
5,1704758400000,2024-01-09,184.035065,42841800,182.822345,184.045000,181.639444,183.239836,45.828301,30.077991,77.328921
6,1704844800000,2024-01-10,185.078812,46792900,183.249796,185.287550,182.822355,184.396239,66.021856,47.395886,103.273795
7,1704931200000,2024-01-11,184.482391,49128400,185.426719,185.933685,182.524147,184.313408,66.142815,59.330991,79.766464
8,1705017600000,2024-01-12,184.810410,40444700,184.949573,185.625523,184.084771,184.840234,69.286710,67.150460,73.559210
9,1705363200000,2024-01-16,182.534088,65603000,181.072860,183.160318,179.850190,181.848199,61.785810,65.738445,53.880539


In [27]:
def exponential_smoothing(a, stock_data):
    """
    Applies exponential smoothing to each column (except timestamp and date) 
    of the dataframe
    0 < alpha < 1
    At alpha = 1, the smoothed data is equal to the initial data
    """
    stock_data[['Close', 'Volume', 'Open', 'High', 'Low']] = stock_data[['Close', 'Volume', 'Open', 'High', 'Low']].ewm(alpha = a, adjust = True).mean()
    return stock_data

In [28]:
def flatten_columns(df):
    """Flattens MultiIndex column names into single strings."""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
    return df

def get_all_features(ticker, start_date, end_date, alpha = 0.8, smoothing = True,
                     macd_short_window=12, macd_long_window=26, macd_signal_window=9, 
                     rsi_window=14, 
                     williams_r_window=14, 
                     window_k_raw=9, window_d=3, window_k_smooth=3):
    """
    Returns a DataFrame with all stock indicators:
    - MACD (Moving Average Convergence Divergence)
    - RSI (Relative Strength Index)
    - Williams %R (Momentum indicator)
    - KDJ (Stock trend analysis)
    """

    basic_data = single_stock_table(ticker, start_date, end_date)
    macd_data = fetch_macd(ticker, start_date, end_date, 
                           macd_short_window, macd_long_window, macd_signal_window)
    rsi_data = fetch_rsi(ticker, start_date, end_date, rsi_window)
    
    # exponentially smooth data
    if smoothing:
        basic_data = exponential_smoothing(alpha, basic_data)

    basic_data = flatten_columns(basic_data)
    macd_data = flatten_columns(macd_data)
    rsi_data = flatten_columns(rsi_data)
    



    basic_data.rename(columns={"Date_": "Date"}, inplace=True)
    macd_data.rename(columns={"Timestamp_": "Date"}, inplace=True)
    rsi_data.rename(columns={"Timestamp_": "Date"}, inplace=True)

    for df in [basic_data, macd_data, rsi_data]:
        if "Date" in df.columns:
            df["Date"] = pd.to_datetime(df["Date"])

    df = pd.merge(basic_data, macd_data, how="inner", on=["Date"])
    df = pd.merge(df, rsi_data, how="inner", on=["Date"])

    high_col = [col for col in df.columns if "High" in col][0]
    low_col = [col for col in df.columns if "Low" in col][0]
    close_col = [col for col in df.columns if "Close" in col][0]

    df.rename(columns={high_col: "High", low_col: "Low", close_col: "Close"}, inplace=True)

    df = calculate_williams_r(df, williams_r_window)

    df.rename(columns={"High": "High_AAPL", "Low": "Low_AAPL", "Close": "Close_AAPL"}, inplace=True)

    df = calculate_KDJ(df, window_k_raw, window_d, window_k_smooth)

    return df.dropna().reset_index(drop=True)


In [29]:
# Example of get_all_features
df = get_all_features('AAPL', '2022-01-01', '2025-02-21')
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Timestamp_,Date,Close_AAPL,Volume_AAPL,Open_AAPL,High_AAPL,Low_AAPL,Vwap_,MACD Value_,Signal_,RSI Value_,Williams %R,%K,%D,%J
0,1641254400000,2022-01-04,176.988057,1.001733e+08,178.703056,179.784138,175.808694,177.481100,-0.181104,-0.036221,0.000000,-54.502002,64.334969,73.753455,45.497998
1,1641340800000,2022-01-05,172.894329,9.562838e+07,176.943480,177.596566,172.444143,173.540011,-0.695687,-0.168114,0.000000,-93.866671,44.934422,64.147111,6.509046
2,1641427200000,2022-01-06,169.807582,9.665051e+07,171.163470,173.340760,169.434621,170.005216,-1.319850,-0.398461,0.000000,-96.396341,18.411662,42.560351,-29.885717
3,1641513600000,2022-01-07,169.328648,8.869484e+07,170.165755,171.583773,168.357549,169.481016,-1.780500,-0.674869,1.669559,-91.501409,6.078526,23.141537,-28.047495
4,1641772800000,2022-01-10,169.248757,1.031524e+08,166.970765,169.943415,165.893688,168.013389,-2.119548,-0.963805,1.862636,-75.846218,12.085344,12.191844,11.872344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
780,1739404800000,2025-02-13,240.389541,5.212231e+07,235.662805,241.162596,234.483118,239.813334,-0.814748,-2.067753,67.753111,-23.268398,56.768903,32.906730,104.493250
781,1739491200000,2025-02-14,243.757913,4.314142e+07,240.132561,244.672522,239.688628,243.713338,0.112820,-1.631638,65.145573,-9.621829,79.469585,54.552454,129.303846
782,1739836800000,2025-02-18,244.327584,4.768628e+07,243.346507,245.078498,241.409723,243.829997,0.827891,-1.139732,57.847549,-6.630878,95.150087,77.129525,131.191210
783,1739923200000,2025-02-19,244.761513,3.530062e+07,244.397304,245.823695,242.809947,244.679998,1.410605,-0.629665,57.120857,-5.509350,95.004602,89.874758,105.264291
